In [ ]:
!pip install infomap
!pip install wurlitzer
!pip install leidenalg 
!pip install git+https://github.com/GiulioRossetti/cdlib.git > /dev/null
#!pip install cigram 

     |████████████████████████████████| 266kB 4.3MB/s 
  Created wheel for infomap: filename=infomap-1.1.3-cp36-cp36m-linux_x86_64.whl size=5682048 sha256=abab7b9ce771c43d45be970148f201f9eb86a6f256f46d0efb009981b6ebffc5
  Stored in directory: /root/.cache/pip/wheels/63/31/ca/9ed0af4c2da472ea989c966c26a6ff448b76f39cb5db82960e
Successfully built infomap
     |████████████████████████████████| 2.4MB 25kB/s 
     |████████████████████████████████| 3.2MB 12.2MB/s 
  Running command git clone -q https://github.com/GiulioRossetti/cdlib.git /tmp/pip-req-build-midprvjm


In [ ]:
from cdlib import algorithms
#from cdlib import viz
from cdlib import NodeClustering
from cdlib import evaluation
import networkx as nx
#from networkx.generators.community import LFR_benchmark_graph
import time
import pandas as pd
import numpy as np
#from cigram import lfr_benchmark_graph
import matplotlib.pyplot as plt
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile

#testing
import signal
from contextlib import contextmanager
import traceback

In [10]:
#*********************** INNER FUNC (GENERIC FUNCTION)*************************************
def evaluate(graph_comms_list, algorithm_dict, eval_method_dict, benchmark, case_name=None):
  
  '''
  Code for Testing 
  '''
  class TimeoutException(Exception): pass

  @contextmanager
  def time_limit(seconds):
    def signal_handler(signum, frame):
      raise TimeoutException("Timed out!")
    signal.signal(signal.SIGALRM, signal_handler)
    signal.alarm(seconds)
    try:
      yield
    finally:
      signal.alarm(0)
  '''
  Code for Testing 
  '''

  if case_name:
    case = case_name
  else:
    case = '-'

  #Check graph_comms_list parameter
  if isinstance(graph_comms_list, list) and len(graph_comms_list):
    graphs = [gc[0] for gc in graph_comms_list] 
    communities = [gc[1] for gc in graph_comms_list]
    attributs = [gc[2] for gc in graph_comms_list]
  elif isinstance(graph_list, tuple):
    graphs = list(graph_comms_list[0])
    communities = list(graph_comms_list[1])
    attributs = list([gc[2] for gc in graph_comms_list]) 
  else:
    raise ValueError("Please insert a list of tuples or a list of lists (graph, commnunities, attrib).")

  #Creation of NodeClustering objects     
  lfr_nodclust_obj = [NodeClustering(C, G, 'Ground Truth') for C, G in zip(communities, graphs)]

  #Application of algorithms over all the benchmarks
  res = pd.DataFrame(columns=['Algorithm', 'Graph', 'Score', 'Metric', 'Benchmark', 'Case', 'Exec_Time', 'Status', 'N', 'Size_C', 'ut', 'Rep'])

  error_log = ""   
  for alg in algorithm_dict:
    algo_data = [] 
    #Check alg
    if isinstance(algorithm_dict[alg], list) or isinstance(algorithm_dict[alg], tuple):
      for G, C, A, func in zip(graphs, lfr_nodclust_obj, attributs, algorithm_dict[alg]):
        #try:
          with time_limit(60):
            s_time = time.time()
            solution = func(G)
            e_time = time.time() - s_time
            for method in eval_method_dict:
              #Performance evaluation 
              perf = eval_method_dict[method](C, solution)
              algo_data.append([alg, A['name'], perf[0], method, benchmark, case, e_time, 'ok', float(A['n'])*1000, A['sz'], float(A['ut']), float(A['rep'])])
        #except Exception as e:          
          #error_log = traceback.format_exc()
          #print(error_log)
          #break
    elif isinstance(algorithm_dict[alg], dict):
      for G, C, A, func in zip(graphs, lfr_nodclust_obj, attributs, algorithm_dict[alg]):
        #try:
          with time_limit(60):
            s_time = time.time()
            solution = algorithm_dict[alg][func](G)
            e_time = time.time() - s_time
            for method in eval_method_dict:
              #Performance evaluation 
              perf = eval_method_dict[method](C, solution)
              algo_data.append([alg, A['name'], perf[0], method, benchmark, case, e_time, 'ok', float(A['n'])*1000, A['sz'], float(A['ut']), float(A['rep'])])
        #except Exception as e:          
          #error_log = traceback.format_exc()
          #print(error_log)
    else: 
      for G, C, A in zip(graphs, lfr_nodclust_obj, attributs):
        #try:
          with time_limit(60):
            s_time = time.time()
            solution = algorithm_dict[alg](G)
            e_time = time.time() - s_time
            for method in eval_method_dict:
              #Performance evaluation 
              perf = eval_method_dict[method](C, solution)
              algo_data.append([alg, A['name'], perf[0], method, benchmark, case, e_time, 'ok', float(A['n'])*1000, A['sz'], float(A['ut']), float(A['rep'])])
        #except Exception as e:
          #algo_data.append([alg, A['name'], float('nan'), '-', benchmark, case, float('nan'), repr(e), float(A['n']), '-', float('nan'), float('nan')])
          #error_log = traceback.format_exc()
          #print(error_log)
          #break
    
    #Update of results
    algo_data_df = pd.DataFrame(algo_data, columns=['Algorithm', 'Graph', 'Score', 'Metric', 'Benchmark', 'Case', 'Exec_Time', 'Status', 'N', 'Size_C', 'ut', 'Rep'])
    #algo_data_df['NMI'] = perf_df['score']
    res = pd.concat([res, algo_data_df], ignore_index=True)

  #Convert some attribute types to numerical
  #res['N'] = res['N']*1000
  #res['ut'] = res['ut'].astype(float)
  #res['Rep'] = res['Rep'].astype(int)

  return res

In [25]:
# *********************************** OUTTER FUNC (STANDARD TEST CASES) *******************************************

def evaluate_and_compare(algos_list, methods_list, benchmark_type, case_list='all'):
  '''
  Some CD algorthms (CDlib) <algos_list>       [CD algorithms to apply] :
    - 'gn'            [girvan_newman]
    - 'greedy'        [greedy_modularity (Clauset et al)]
    - 'louvain'       [louvain (Blondel et al)]
    - 'cfinder'       [kclique]
    - 'mcl'           [markov_clustering]
    - 'infomap'       [infomap]
    - 'em'            [expectation-maximization]
    - 'leiden'        [leiden]
  
  Partition evaluation methods (CDlib) <methods_list>       [evaluation metric to apply] :
    - 'nmi'           [Normalized Mutual Information]
    - 'ami'           [Adjusted Mutual Information]
    - 'ari'           [Adjusted rand index]
    - 'f1'            [F1 score]
    - 'nf1'           [Normalized F1 score]
    - 'omega'         [Index of resemblance for overlapping]
    - 'onmi_lfk'      [Overlapping Normalized Mutual Information 1]
    - 'onmi_mgh'      [Overlapping Normalized Mutual Information 2]
    - 'var_inf'       [Variation of Information]

  Possible values for <benchmark_type>       [benchmark to evaluate] :
    - 'lfr_undir_unwei'
    - 'lfr_undir_wei'
    - 'lfr_dir_unwei'
    - 'lfr_undir_unwei_ovlp'

  Possible values for <case_list>       [benchmark cases to evaluate] :
    - 'n_1k_small'     [n=1000, minc=10, maxc=50]
    - 'n_1k_big'       [n=1000, minc=10, maxc=50] 
    - 'n_5k_small'     [n=5000, minc=20, maxc=100]
    - 'n_5k_big'       [n=5000, minc=20, maxc=100]
    - 'all'           [perform evaluation over all 4 cases]

    * It is possible to select multiple cases passing them as a list.
  '''
  
  start_time = time.time()
  
  #CD Algorithms names list (Crisp Communities)
  algos_dict = {
    # Algos present in paper  
    'gn'          : algorithms.girvan_newman,
    'greedy'      : algorithms.greedy_modularity,
    'louvain'     : algorithms.louvain,
    'cfinder'     : algorithms.kclique,
    'mcl'         : algorithms.markov_clustering,
    'infomap'     : algorithms.infomap,
    'em'          : algorithms.em,
    'leiden'      : algorithms.leiden,

    # The others algos in CDlib
    'agdl'        : algorithms.agdl,
    'aslpaw'      : algorithms.aslpaw,
    'async_fluid' : algorithms.async_fluid,
    'cpm'         : algorithms.cpm,
    'der'         : algorithms.der,
    'edmot'       : algorithms.edmot,
    'eigenvector' : algorithms.eigenvector,
    'gdmp2'       : algorithms.gdmp2,
    'label_prop'  : algorithms.label_propagation,
    'rber_pots'   : algorithms.rber_pots,
    'rb_pots'     : algorithms.rb_pots,
    'scan'        : algorithms.scan,
    'signif_com'  : algorithms.significance_communities,
    'spinglass'   : algorithms.spinglass,
    'surp_com'    : algorithms.surprise_communities,
    'walktrap'    : algorithms.walktrap,
    'sbm_dl'      : algorithms.sbm_dl,
    'sbm_dl_nstd' : algorithms.sbm_dl_nested       
  }

  #Partition comparisons scores (CDlib)
  methods_dict = {
    'nmi'      : evaluation.normalized_mutual_information,
    'ami'      : evaluation.adjusted_mutual_information,
    'ari'      : evaluation.adjusted_rand_index,
    'f1'       : evaluation.f1,
    'nf1'      : evaluation.nf1,
    'omega'    : evaluation.omega,
    'onmi_lfk' : evaluation.overlapping_normalized_mutual_information_LFK,
    'onmi_mgh' : evaluation.overlapping_normalized_mutual_information_MGH,
    'vi'       : evaluation.variation_of_information
  }

  #Importing and reading benchmark files

  #Url base path
  url = 'https://github.com/CarlosVargasF/LFR_benchmarks_for_testing/raw/fac7461a1e2132e62149d211589ac36e5093ff6a/'
  
  ''' #Folders for undirected and unweighted benchmarks
  src_undir_unwei = {
    'n_1k_sz_small' : 'lfr_n1000_small/',
    'n_1k_sz_big'   : 'lfr_n1000_big/',
    'n_5k_sz_small' : 'lfr_n5000_small/',
    'n_5k_sz_big'   : 'lfr_n5000_big/'
  }

  #Folders for undirected and weighted benchmarks
  src_undir_wei = {
    'n_1k_sz_small'  : 'lfr_n_small_mut05/',
    'n_1k_sz_big'    : 'lfr_n5000_small_mut08/',
    'n_5k_sz_small'  : 'lfr_n5000_big_mut05/',
    'n_5k_sz_big'    : 'lfr_n5000_big_mut08/'
  }

   #Folders for directed and unweighted benchmarks
  src_dir_unwei = {
    'n_1k_sz_small' : 'lfr_n1000_small/',
    'n_1k_sz_big'   : 'lfr_n1000_big/',
    'n_5k_sz_small' : 'lfr_n5000_small/',
    'n_5k_sz_big'   : 'lfr_n5000_big/'
  }

  #Folders for undirected and weighted benchmarks
  src_undir_wei = {
    'n_5k_sz_small_mut01' : 'lfr_n5000_small_mut01/',
    'n_5k_sz_small_mut03' : 'lfr_n5000_small_mut03/',
    'n_5k_sz_big_mut01'   : 'lfr_n5000_big_mut01/',
    'n_5k_sz_big_mut03'   : 'lfr_n5000_big_mut03/'
  } '''

  #Graph type
  #parameters for graph generation
  data_edg=False
  create_using=nx.Graph
  #Select file folders and adjust some parameters
  if isinstance(benchmark_type, str) and len(benchmark_type):
    if benchmark_type == 'undir_unwei':
      url = url + 'undirected_unweighted/lfr_'            
    elif benchmark_type == 'undir_wei':
      url = url + 'undirected_weighted/lfr_'
      data_edg=(('weight',float),)
    elif benchmark_type == 'dir_unwei':
      url = url + 'directed_unweighted/lfr_'
      create_using=nx.DiGraph
    elif benchmark_type == 'undir_unwei_ovlp':
      url = url + 'undirected_unweighted_overlap/lfr_'
    else:
      raise ValueError('benchmark_type not supported')
  else:
    raise TypeError('Insert a valide benchmark_type as a string')

  #Checking case parameter  
  if isinstance(case_list, str): 
    if case_list == 'all':
      cases = [opt for opt in src]
    else:
      cases = list(case_list.split())
  elif (isinstance(case_list, list)) and (len(case_list)): 
    for c in case_list:
      if not(c in src):
        raise ValueError('Invalid <case> option. Verify available cases for the selected benchmark.')
    cases = case_list    
  else:
    raise TypeError('Invalid <case> input format. Please insert a list or a space-separated string of valide options or "all" for use all of them.')
  
  #Checking algos_list parameter
  if (isinstance(algos_list, str)):
    for a in list(algos_list.split()):
      if not(a in algos_dict):
        raise ValueError('Invalid algos_dict key. Verify available algorithm names.')
    algos = dict((k, v) for k, v in algos_dict.items() if k in list(algos_list.split()))
    
  elif (isinstance(algos_list, list)) and (len(algos_list)): 
    for a in algos_list:
      if not(a in algos_dict):
        raise ValueError('Invalid algos_dict key. Verify available algorithm names.')
    algos = dict((k, v) for k, v in algos_dict.items() if k in algos_list)    
  else:
    raise TypeError('Invalid <algos_list> input format. Please insert a list or a space-separated string of valide algorithm names.') 

  #Checking methods_list parameter
  if (isinstance(methods_list, str)):
    methods = dict((k, v) for k, v in methods_dict.items() if k in list(methods_list.split()))
  elif (isinstance(methods_list, list)) and (len(methods_list)): 
    for m in methods_list:
      if not(m in methods_dict):
        raise ValueError('Invalid methods_dict key. Verify available evaluation method names.')
    methods = dict((k, v) for k, v in methods_dict.items() if k in methods_list)    
  else:
    raise TypeError('Invalid <methods_list> input format. Please insert a list or a space-separated string of evaluation method names.') 

  #Generate graphs according to cases   
  results = pd.DataFrame(columns=['Algorithm', 'Graph', 'Score', 'Metric', 'Benchmark', 'Case', 'Exec_Time', 'Status', 'N', 'Size_C', 'ut', 'Rep'])
  for case in cases:
    base_path = url + case 

    #Open zip files
    zip_comms = urlopen(base_path + '/cnl_files.zip')
    zip_edges = urlopen(base_path + '/nse_files.zip')
    zipfile_c = ZipFile(BytesIO(zip_comms.read()))
    zipfile_e = ZipFile(BytesIO(zip_edges.read()))

    #Read data files
    data_comms = [zipfile_c.open(line1).read().decode('utf-8').splitlines() for line1 in zipfile_c.namelist()]
    data_edges = [(zipfile_e.open(line2), line2) for line2 in zipfile_e.namelist()]
        
    #Construction of the lfr graphs

    #List of graphs
    lfr_graphs = []
    #List of graph_attribute dictionaries 
    graph_attributes = []

    for edge_file in data_edges:
      attrib_dict = {}
      g = nx.read_edgelist(edge_file[0], nodetype=int, data=data_edg, create_using=create_using)
      #g.name = edge_file[1]

      # case --> (name, n, size_comms) / edge_file --> (ut, rep, [uw, ovlp_fraction])
      attribs_1 = case.split('_')
      attribs_2 = edge_file[1].split('_')
      
      attrib_dict.update([('name', edge_file[1][:-4]),('n', attribs_1[attribs_1.index('n')+1][:-1]), ('sz', attribs_1[attribs_1.index('sz')+1]),
                          ('ut', attribs_2[attribs_2.index('ut')+1]), ('rep', attribs_2[attribs_2.index('rep')+1][:-4])])
      lfr_graphs.append(g)
      graph_attributes.append(attrib_dict)

    #Construction of the lfr communities
    lfr_comms = [[list(map(int, item.split())) for item in comm_file] for comm_file in data_comms]

    #Creation of specific functions for some algoritmhs with aditional parameters
    #def gn(G) : algorithms.girvan_newman(G,k)
    if 'gn' in algos:
      def make_gn(i):
        def gn(G): return algorithms.girvan_newman(G,i)
        return gn
      algos['gn'] = [make_gn(len(c)) for c in lfr_comms] 

    if 'em' in algos:
      def make_em(i):
        def em(G): return algorithms.em(G,i)
        return em
      algos['em'] = [make_em(len(c)) for c in lfr_comms]

    if 'async_fluid' in algos:
      def make_af(i):
        def af(G): return algorithms.em(G,i)
        return af
      algos['async_fluid'] = [make_af(len(c)) for c in lfr_comms]

    if 'cfinder' in algos:
      def make_cfinder(i):
        def cfinder(G): return algorithms.kclique(G,i)
        return cfinder
      if attribs_1[attribs_1.index('sz')+1] == 'small': 
        k = 10
      else: 
        k = 20
      algos['cfinder'] = make_cfinder(k)   
    #algos['gn'] = [gn(len(c)) for c in lfr_comms]

    #Formating for <evaluate> function 
    graphs_comms_atts = [(g, c, a) for g, c, a in zip(lfr_graphs, lfr_comms, graph_attributes)]

   
    #----------INNER FUNC CALL--------------------------------------------------

    #Apply selected CD algorithms and evaluate them according to selected methods
    res_eval = evaluate(graphs_comms_atts, algos, methods, benchmark_type, case)

    #----------INNER FUNC CALL--------------------------------------------------

    results = pd.concat([results, res_eval], ignore_index=True)

  total_time = (time.time() - start_time)
  print("--- Total execution time: %d min %d sec ---" % (total_time//60, total_time%60))

  return results 


# Agorithm Test Report

## - Algorithms presented in paper

In [ ]:
# Girvan and Newman --------------------------------------------------------------- OK
p=evaluate_and_compare('gn', 'nmi', 'undir_unwei', 'n_1k_sz_small')

In [12]:
# Greedy ------------------------------------------------------------------------- OK
p=evaluate_and_compare('greedy', 'nmi', 'undir_unwei', 'n_1k_sz_small')

--- Total execution time: 3 min 51 sec ---


In [14]:
# Louvain ------------------------------------------------------------------------- OK
p=evaluate_and_compare('louvain', 'nmi', 'undir_unwei', 'n_1k_sz_small')

--- Total execution time: 0 min 46 sec ---


In [11]:
# Cfinder ------------------------------------------------------------------------- ERR
p=evaluate_and_compare('cfinder', 'nmi', 'undir_unwei', 'n_1k_sz_small')

ValueError: ignored

In [16]:
# Markov Clustering -------------------------------------------------------------- ERR
p=evaluate_and_compare('mcl', 'nmi', 'undir_unwei', 'n_1k_sz_small')

TimeoutException: ignored

In [15]:
# Infomap ------------------------------------------------------------------------- OK
p=evaluate_and_compare('infomap', 'nmi', 'undir_unwei', 'n_1k_sz_small')

--- Total execution time: 0 min 19 sec ---


In [23]:
# Expectation - Maximization ------------------------------------------------------ ERR
p=evaluate_and_compare('em', 'nmi', 'undir_unwei', 'n_1k_sz_small')

NetworkXError: ignored

In [18]:
# Leiden ------------------------------------------------------------------------- OK
p=evaluate_and_compare('leiden', 'nmi', 'undir_unwei', 'n_1k_sz_small')

--- Total execution time: 0 min 17 sec ---


## - Other algorithms in CDlib

In [ ]:
# Agdl --------------------------------------------------------------------------- ??
p=evaluate_and_compare('', 'nmi', 'undir_unwei', 'n_1k_sz_small')

In [24]:
# ASLPAw ------------------------------------------------------------------------- ERR
p=evaluate_and_compare('aslpaw', 'nmi', 'undir_unwei', 'n_1k_sz_small')

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_iforest.py:248: FutureWarning: 'behaviour' is deprecated in 0.22 and will be removed in 0.24. You should not pass or set this parameter.
  FutureWarning
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_iforest.py:248: FutureWarning: 'behaviour' is deprecated in 0.22 and will be removed in 0.24. You should not pass or set this parameter.
  FutureWarning
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_iforest.py:248: FutureWarning: 'behaviour' is deprecated in 0.22 and will be removed in 0.24. You should not pass or set this parameter.
  FutureWarning
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_iforest.py:248: FutureWarning: 'behaviour' is deprecated in 0.22 and will be removed in 0.24. You should not pass or set this parameter.
  FutureWarning
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_iforest.py:248: FutureWarning: 'behaviour' is deprecated in 0.22 and will be removed in 0.24. You shoul

TimeoutException: ignored

In [26]:
# Async fluid ------------------------------------------------------------------- ERR
p=evaluate_and_compare('async_fluid', 'nmi', 'undir_unwei', 'n_1k_sz_small')

NetworkXError: ignored

In [27]:
# Cpm ------------------------------------------------------------------------- OK
p=evaluate_and_compare('cpm', 'nmi', 'undir_unwei', 'n_1k_sz_small')

--- Total execution time: 0 min 16 sec ---


In [29]:
# Der ------------------------------------------------------------------------- OK
p=evaluate_and_compare('der', 'nmi', 'undir_unwei', 'n_1k_sz_small')

/usr/local/lib/python3.6/dist-packages/cdlib/algorithms/internal/DER.py:140: RuntimeWarning: divide by zero encountered in log
  log_param = np.log(param)
/usr/local/lib/python3.6/dist-packages/cdlib/algorithms/internal/DER.py:140: RuntimeWarning: divide by zero encountered in log
  log_param = np.log(param)
/usr/local/lib/python3.6/dist-packages/cdlib/algorithms/internal/DER.py:140: RuntimeWarning: divide by zero encountered in log
  log_param = np.log(param)
/usr/local/lib/python3.6/dist-packages/cdlib/algorithms/internal/DER.py:140: RuntimeWarning: divide by zero encountered in log
  log_param = np.log(param)
/usr/local/lib/python3.6/dist-packages/cdlib/algorithms/internal/DER.py:140: RuntimeWarning: divide by zero encountered in log
  log_param = np.log(param)
/usr/local/lib/python3.6/dist-packages/cdlib/algorithms/internal/DER.py:140: RuntimeWarning: divide by zero encountered in log
  log_param = np.log(param)
/usr/local/lib/python3.6/dist-packages/cdlib/algorithms/internal/DER.p

--- Total execution time: 0 min 48 sec ---


In [31]:
# Edmot ------------------------------------------------------------------------- ERR
p=evaluate_and_compare('edmot', 'nmi', 'undir_unwei', 'n_1k_sz_small')

AssertionError: ignored

In [32]:
# Eigenvector -------------------------------------------------------------------- ERR
p=evaluate_and_compare('eigenvector', 'nmi', 'undir_unwei', 'n_1k_sz_small')

/usr/local/lib/python3.6/dist-packages/igraph/__init__.py:1153: RuntimeWarning: ARPACK solver failed to converge (3001 iterations, 0/1 eigenvectors converged) at /project/vendor/source/igraph/src/arpack.c:819
  membership, _, q = GraphBase.community_leading_eigenvector(self, clusters, **kwds)


InternalError: ignored

In [33]:
# Gdmp2 ------------------------------------------------------------------------- ERR
p=evaluate_and_compare('gdmp2', 'nmi', 'undir_unwei', 'n_1k_sz_small')

ValueError: ignored

In [34]:
# Label_prop ------------------------------------------------------------------------- OK
p=evaluate_and_compare('label_prop', 'nmi', 'undir_unwei', 'n_1k_sz_small')

--- Total execution time: 0 min 12 sec ---


In [36]:
# Rber_pots ------------------------------------------------------------------------- OK
p=evaluate_and_compare('rber_pots', 'nmi', 'undir_unwei', 'n_1k_sz_small')

--- Total execution time: 0 min 16 sec ---


In [38]:
# Rb_pots ---------------------------------------------------------------------------- OK
p=evaluate_and_compare('rb_pots', 'nmi', 'undir_unwei', 'n_1k_sz_small')

--- Total execution time: 0 min 15 sec ---


In [ ]:
# Scan (Structural clustering algo) ------------------------------------------------ ??
p=evaluate_and_compare('scan', 'nmi', 'undir_unwei', 'n_1k_sz_small')

In [39]:
# Significance communities ----------------------------------------------------------- OK
p=evaluate_and_compare('signif_com', 'nmi', 'undir_unwei', 'n_1k_sz_small')

--- Total execution time: 0 min 20 sec ---


In [42]:
# Spinglass ------------------------------------------------------------------------- ERR
p=evaluate_and_compare('spinglass', 'nmi', 'undir_unwei', 'n_1k_sz_small')

InternalError: ignored

In [43]:
# Surprise_communities ------------------------------------------------------------- OK
p=evaluate_and_compare('surp_com', 'nmi', 'undir_unwei', 'n_1k_sz_small')

--- Total execution time: 0 min 18 sec ---


In [44]:
# Walktrap ------------------------------------------------------------------------- OK
p=evaluate_and_compare('walktrap', 'nmi', 'undir_unwei', 'n_1k_sz_small')

--- Total execution time: 0 min 25 sec ---


In [45]:
# Sbm_dl (stockastic block model)---------------------------------------------------- ERR (graph-tool) 
p=evaluate_and_compare('sbm_dl', 'nmi', 'undir_unwei', 'n_1k_sz_small')

Exception: ignored

In [46]:
# Sbm dl nested (stockastic block model)----------------------------------------- ERR (graph-tool) 
p=evaluate_and_compare('sbm_dl_nstd', 'nmi', 'undir_unwei', 'n_1k_sz_small')

Exception: ignored